In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten,Dense
from keras.losses import CategoricalCrossentropy
from keras.optimizers import Adam
from keras.metrics import Precision, Recall, AUC

In [2]:
#讀取數據集
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00236/seeds_dataset.txt"
column_names = ['area', 'perimeter', 'compactness', 'length_of_kernel', 'width_of_kernel', 'asymmetry_coefficient', 'length_of_kernel_groove', 'class']
df = pd.read_csv(url, sep='\s+', header=None, names=column_names)

In [3]:
#分割特徵和標籤
X = df.drop('class', axis=1)
y = df['class']

#將標籤進行one-hot編碼
y = to_categorical(y - 1)

#正規化特徵
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [4]:
#分割為訓練、測試和驗證集
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=64, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

#將數據轉換為3D張量 (用於Conv1D層)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = X_val[:32].reshape((32, X_val.shape[1], 1))  # 將驗證集改為 32 比
X_test = X_test[:32].reshape((32, X_test.shape[1], 1))  # 將測試集改為 32 比
print("訓練集樣本數量:", len(X_train))
print("驗證集樣本數量:", len(X_val))
print("測試集樣本數量:", len(X_test))

訓練集樣本數量: 146
驗證集樣本數量: 32
測試集樣本數量: 32


In [5]:
#建立CNN模型
model = Sequential()
model.add(Conv1D(1, kernel_size=1,activation='relu',padding='same',input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(loss=CategoricalCrossentropy(),optimizer=Adam(learning_rate=0.006414053261659062), metrics=['accuracy', Precision(), Recall(), AUC()])

model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_data=(X_val, y_val[:32]))

Epoch 1/1000
5/5 [==============================] - 2s 106ms/step - loss: 0.9577 - accuracy: 0.4041 - precision: 0.9167 - recall: 0.3014 - auc: 0.6693 - val_loss: 0.8972 - val_accuracy: 0.4062 - val_precision: 0.9091 - val_recall: 0.3125 - val_auc: 0.7268
Epoch 2/1000
5/5 [==============================] - 0s 14ms/step - loss: 0.8726 - accuracy: 0.5548 - precision: 0.8846 - recall: 0.3151 - auc: 0.8128 - val_loss: 0.8257 - val_accuracy: 0.5625 - val_precision: 0.9091 - val_recall: 0.3125 - val_auc: 0.8408
Epoch 3/1000
5/5 [==============================] - 0s 12ms/step - loss: 0.7979 - accuracy: 0.6712 - precision: 0.8889 - recall: 0.3288 - auc: 0.8765 - val_loss: 0.7628 - val_accuracy: 0.7500 - val_precision: 0.8333 - val_recall: 0.3125 - val_auc: 0.8801
Epoch 4/1000
5/5 [==============================] - 0s 12ms/step - loss: 0.7295 - accuracy: 0.7329 - precision: 0.8889 - recall: 0.3288 - auc: 0.9087 - val_loss: 0.7077 - val_accuracy: 0.8125 - val_precision: 0.9091 - val_recall: 0.31

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 7, 1)              2         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 7, 1)             0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 7)                 0         
                                                                 
 dense (Dense)               (None, 10)                80        
                                                                 
 dense_1 (Dense)             (None, 3)                 33        
                                                                 
Total params: 115
Trainable params: 115
Non-trainable params: 0
__________________________________________________________

In [7]:
result_train = model.evaluate(X_train,y_train)
result_train_f = round(result_train[1],2)
print(result_train_f)

5/5 [==============================] - 0s 6ms/step - loss: 0.0136 - accuracy: 0.9932 - precision: 0.9932 - recall: 0.9932 - auc: 1.0000
0.99


In [8]:
result_test = model.evaluate(X_test,y_test)
result_test_f = round(result_test[1],2)
print(result_test_f)

1/1 [==============================] - 0s 27ms/step - loss: 0.1254 - accuracy: 0.9375 - precision: 0.9375 - recall: 0.9375 - auc: 0.9971
0.94


In [9]:
result_val = model.evaluate(X_val,y_val)
result_val_f = round(result_val[1],2)
print(result_val_f)

1/1 [==============================] - 0s 24ms/step - loss: 0.2512 - accuracy: 0.9688 - precision: 0.9688 - recall: 0.9688 - auc: 0.9836
0.97
